## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-05-09-00-00 +0000,cbc,Carney heads to Paris as allies discuss securi...,https://www.cbc.ca/news/politics/ukraine-russi...
1,2026-01-05-09-00-00 +0000,cbc,Canadian airlines could be forced to 'up their...,https://www.cbc.ca/news/politics/expanded-air-...
2,2026-01-05-08-55-59 +0000,bbc,Gold price rises after US captures Venezuela's...,https://www.bbc.com/news/articles/crrnw08qvg7o...
3,2026-01-05-08-55-00 +0000,wsj,U.S. Actions in Venezuela Unlikely to Shift Gl...,https://www.wsj.com/finance/commodities-future...
4,2026-01-05-08-53-00 +0000,wsj,Investors Could Favor Dollar Amid Geopolitical...,https://www.wsj.com/finance/currencies/markets...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2342/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
23,venezuela,61
123,trump,38
24,maduro,37
116,new,24
88,leader,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
129,2026-01-04-20-33-28 +0000,nypost,Trump threatens Venezuela’s new leader Delcy R...,https://nypost.com/2026/01/04/us-news/trump-th...,185
171,2026-01-04-15-22-48 +0000,nyt,How Trump Fixed On a Maduro Loyalist as Venezu...,https://www.nytimes.com/2026/01/04/world/ameri...,174
35,2026-01-05-04-57-44 +0000,nypost,"Sara Foster praises Trump’s capture of Maduro,...",https://nypost.com/2026/01/04/us-news/sara-fos...,154
21,2026-01-05-07-34-13 +0000,wapo,Live updates: Trump says U.S. is ‘in charge’ i...,https://www.washingtonpost.com/world/2026/01/0...,143
166,2026-01-04-16-54-24 +0000,nyt,Trump Long Wanted to ‘Take the Oil.’ He Says H...,https://www.nytimes.com/2026/01/03/us/politics...,141


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
129,185,2026-01-04-20-33-28 +0000,nypost,Trump threatens Venezuela’s new leader Delcy R...,https://nypost.com/2026/01/04/us-news/trump-th...
176,48,2026-01-04-14-45-20 +0000,nypost,Marco Rubio says Venezuela’s VP Delcy Rodrígue...,https://nypost.com/2026/01/04/us-news/marco-ru...
138,48,2026-01-04-19-53-22 +0000,nypost,Owner of Swiss nightclub where deadly fire bro...,https://nypost.com/2026/01/04/world-news/owner...
68,42,2026-01-05-01-24-33 +0000,nypost,"6-year-old girl, 90-year-old passenger killed ...",https://nypost.com/2026/01/04/us-news/6-year-o...
211,37,2026-01-04-10-00-00 +0000,wsj,There are 35 Senate seats up for election in 2...,https://www.wsj.com/politics/elections/democra...
157,34,2026-01-04-18-09-40 +0000,nypost,Satellite images reveal damage from US strikes...,https://nypost.com/2026/01/04/world-news/satel...
70,29,2026-01-05-01-05-44 +0000,nypost,Far-left extremist attack on Berlin power grid...,https://nypost.com/2026/01/04/world-news/far-l...
97,27,2026-01-04-23-19-44 +0000,nypost,Freed Israeli hostage details fear of being ma...,https://nypost.com/2026/01/04/world-news/freed...
126,25,2026-01-04-20-40-24 +0000,nypost,OPEC+ keeps oil output steady amid turmoil amo...,https://nypost.com/2026/01/04/business/opec-ke...
61,23,2026-01-05-02-00-00 +0000,wsj,Trump’s Venezuela incursion has injected a vol...,https://www.wsj.com/politics/policy/trump-is-t...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
